# Review Data Loading

## Introduction

For my capstone project, I will be using machine learning on the Yelp Open Dataset to predict the rating of a restaurant given its characteristics. The motivation behind building a restaurant rating predictor is to help future restaurant owners anticipate the requirements that a restaurant must meet in order to receive a desirable star rating and achieve success for the business.

## Methodology

In this notebook, we will load the required JSON files from the Yelp Open dataset, keeping only the required records that pertain to restaurants in Massachusetts, and export the data into CSV files for the next step in this project which is data cleaning and EDA.

## Import the Required Libraries

We will require the Pandas and JSON libraries for this notebook.

In [1]:
import pandas as pd
import json

## Load the Business Dataset

We will begin by loading business dataset JSON file into a Pandas data frame.

In [4]:
%%time
business_df = pd.read_json('../data/business.json', lines=True)
business_df.head()

CPU times: user 2.84 s, sys: 717 ms, total: 3.56 s
Wall time: 3.63 s


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


We can see that there is a `categories` column that includes string descriptions of the business category. We will filter for the word *'restaurant'* which is the category of interest for this project.

In [5]:
# Filter for restaurnts that contain a restauarant description in the categories column
# and save to new dataframe
restaurant_df = business_df[business_df['categories'].str.contains('restaurant', case=False)==True].reset_index().drop('index', axis=1)
restaurant_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,D4JtQNTI4X3KcbzacDJsMw,Bob Likes Thai Food,3755 Main St,Vancouver,BC,V5V,49.251342,-123.101333,3.5,169,1,"{'GoodForKids': 'True', 'Alcohol': 'u'none'', ...","Restaurants, Thai","{'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'..."
3,jFYIsSb7r1QeESVUnXPHBw,Boxwood Biscuit,740 S High St,Columbus,OH,43206,39.947007,-82.997471,4.5,11,1,None,"Breakfast & Brunch, Restaurants","{'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0'}"
4,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,42.541155,-70.973438,4.0,39,1,"{'RestaurantsGoodForGroups': 'True', 'HasTV': ...","Food, Pizza, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."


In [6]:
restaurant_df.shape

(50793, 14)

The resulting shape of our restaurant data frame is 50,793 rows and 14 columns.

We now look at the value counts for the different states in this data frame.

In [7]:
restaurant_df['state'].value_counts()

MA     10551
FL      7711
BC      7508
OR      7402
GA      6142
TX      5452
OH      4380
CO       866
WA       774
KS         1
MN         1
VA         1
WY         1
KY         1
NH         1
ABE        1
Name: state, dtype: int64

We can see that the state of Massachusetts (abbreviated as MA) has the most restaurants in the dataset. We will now filter for only Massachusetts restaurants.

In [8]:
# Extract restaurants in Massachusetts
restaurant_df = restaurant_df[restaurant_df['state']=='MA'].reset_index(drop=True)

# Delete the business data frame from memory
del business_df

# View the info summary
restaurant_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10551 entries, 0 to 10550
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   10551 non-null  object 
 1   name          10551 non-null  object 
 2   address       10551 non-null  object 
 3   city          10551 non-null  object 
 4   state         10551 non-null  object 
 5   postal_code   10551 non-null  object 
 6   latitude      10551 non-null  float64
 7   longitude     10551 non-null  float64
 8   stars         10551 non-null  float64
 9   review_count  10551 non-null  int64  
 10  is_open       10551 non-null  int64  
 11  attributes    10481 non-null  object 
 12  categories    10551 non-null  object 
 13  hours         8701 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 1.1+ MB


## Load the Reviews Dataset

We now read in the reviews dataset JSON file into a Pandas data frame. Since the JSON file is large (6.94 GB), we will read the file contents in chunks to reduce the data loading time, as proposed by Eve Law in this blog post:
https://towardsdatascience.com/load-yelp-reviews-or-other-huge-json-files-with-ease-ad804c2f1537

We will only add the records for businesses that are present in the restaurant data frame.

In [7]:
%%time
data_lines = []

with open("../data/review.json", "r") as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                          chunksize=1000)
        
    for chunk in reader:
        # Only include records for businesses in restaurant data frame
        chunk = chunk[chunk['business_id'].isin(restaurant_df['business_id'])]
        data_lines.append(chunk)
    
review_df = pd.concat(data_lines, ignore_index=True)

CPU times: user 2min 17s, sys: 5.73 s, total: 2min 23s
Wall time: 2min 23s


In [8]:
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,J4a2TuhDasjn2k3wWtHZnQ,RNm_RWkcd02Li2mKPRe7Eg,xGXzsc-hzam-VArK6eTvtw,1,2,0,0,"This place used to be a cool, chill place. Now...",2018-01-21 04:41:03
2,28gGfkLs3igtjVy61lh77Q,Q8c91v7luItVB0cMFF_mRA,EXOsmAB1s71WePlQk0WZrA,2,0,0,0,"The setting is perfectly adequate, and the foo...",2006-04-16 02:58:44
3,KKVFopqzcVfcubIBxmIjVA,99RsBrARhhx60UnAC4yDoA,EEHhKSxUvJkoPSzeGKkpVg,5,0,0,0,I work in the Pru and this is the most afforda...,2014-05-07 18:10:21
4,btNWW2kdJYfwpTDyzJO3Iw,DECuRZwkUw8ELQZfNGef2Q,zmZ3HkVCeZPBefJJxzdJ7A,4,0,0,0,Nothing special but good enough. I like anoth...,2012-12-04 04:29:47


In [9]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1365575 entries, 0 to 1365574
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   review_id    1365575 non-null  object        
 1   user_id      1365575 non-null  object        
 2   business_id  1365575 non-null  object        
 3   stars        1365575 non-null  int64         
 4   useful       1365575 non-null  int64         
 5   funny        1365575 non-null  int64         
 6   cool         1365575 non-null  int64         
 7   text         1365575 non-null  object        
 8   date         1365575 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 93.8+ MB


## Load the User Dataset

We now read in the user dataset. This file is also large at 3.68 GB, so we will once again read in its contents in chunks.

In [10]:
%%time
data_lines = []

with open("../data/user.json", "r") as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                         chunksize=1000)
        
    for chunk in reader:
        data_lines.append(chunk)
    
user_df = pd.concat(data_lines, ignore_index=True)

CPU times: user 50.5 s, sys: 3.7 s, total: 54.2 s
Wall time: 54.3 s


In [11]:
user_df.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,163,190,361,147,1212,5691,2541,2541,815,323
1,dIIKEfOgo0KqUfGQvGikPg,Gabi,2136,2007-08-10 19:01:51,21272,10289,18046,"2007,2008,2009,2010,2011,2012,2013,2014,2015,2...","XPzYf9_mwG2eXYP2BAGSTA, 2LooM5dcIk2o01nftYdPIg...",1025,...,87,94,232,96,1187,3293,2205,2205,472,294
2,D6ErcUnFALnCQN4b1W_TlA,Jason,119,2007-02-07 15:47:53,188,128,130,"2010,2011","GfB6sC4NJQvSI2ewbQrDNA, jhZtzZNNZJOU2YSZ6jPlXQ...",16,...,1,3,0,0,5,20,31,31,3,1
3,JnPIjvC0cmooNDfsa9BmXg,Kat,987,2009-02-09 16:14:29,7234,4722,4035,"2009,2010,2011,2012,2013,2014","HQZPQhKMwRAyS6BCselVWQ, kP2U1s_sjQfHO9grxiyDTA...",420,...,129,93,219,90,1120,4510,1566,1566,391,326
4,37Hc8hr3cw0iHLoPzLK6Ow,Christine,495,2008-03-03 04:57:05,1577,727,1124,"2009,2010,2011","-Q88pZUcrfN0BLBDp-bkAQ, etPn4Pv1Gc4cRZjRgB_BOw...",47,...,19,32,16,15,77,131,310,310,98,44


In [27]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2189457 entries, 0 to 2189456
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 367.5+ MB


We now filter for only the users who are present in our review data frame, which mean that they are users who have left reviews for restaurants in Massachusetts.

In [28]:
user_df = user_df[user_df['user_id'].isin(review_df['user_id'])].reset_index(drop=True)
user_df.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,163,190,361,147,1212,5691,2541,2541,815,323
1,JnPIjvC0cmooNDfsa9BmXg,Kat,987,2009-02-09 16:14:29,7234,4722,4035,"2009,2010,2011,2012,2013,2014","HQZPQhKMwRAyS6BCselVWQ, kP2U1s_sjQfHO9grxiyDTA...",420,...,129,93,219,90,1120,4510,1566,1566,391,326
2,37Hc8hr3cw0iHLoPzLK6Ow,Christine,495,2008-03-03 04:57:05,1577,727,1124,"2009,2010,2011","-Q88pZUcrfN0BLBDp-bkAQ, etPn4Pv1Gc4cRZjRgB_BOw...",47,...,19,32,16,15,77,131,310,310,98,44
3,1jXmzuIFKxTnEnR0pxO0Hg,Clara,299,2010-10-01 17:29:36,381,106,121,"2010,2011,2012,2013,2014,2015","VGfzq5na6LZUwxwWO5eVLA, 35uHDsVOEsWbLdEg8Ttobg...",23,...,8,2,6,0,17,47,30,30,4,1
4,cxS6dbjyPgPS1S890u_khA,Joe,65,2005-07-22 11:07:20,116,66,77,,"ISN91YL7nkU_X_bovO2Lyg, sxsXw8QrbFTUzVVGGLNdOg...",2,...,2,0,3,1,1,2,2,2,1,0


In [29]:
user_df.shape

(377202, 22)

After filtering for users who only reviewed restaurants in Massachusetts, the resulting data frame has 377,202 users with 22 columns.

## Load the Check-In dataset

We now read in the check-in dataset.

In [30]:
%%time
checkin_df = pd.read_json('../data/checkin.json', lines=True)
checkin_df.head()

CPU times: user 1.91 s, sys: 888 ms, total: 2.8 s
Wall time: 2.8 s


,business_id,date
0,--0r8K_AQ4FZfLsX3ZYRDA,2017-09-03 17:13:59
1,--0zrn43LEaB4jUWTQH_Bg,"2010-10-08 22:21:20, 2010-11-01 21:29:14, 2010..."
2,--164t1nclzzmca7eDiJMw,"2010-02-26 02:06:53, 2010-02-27 08:00:09, 2010..."
3,--2aF9NhXnNVpDV0KS3xBQ,"2014-11-03 16:35:35, 2015-01-30 18:16:03, 2015..."
4,--2mEJ63SC_8_08_jGgVIg,"2010-12-15 17:10:46, 2013-12-28 00:27:54, 2015..."


In [31]:
checkin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138876 entries, 0 to 138875
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  138876 non-null  object
 1   date         138876 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


We will filter for the data frame to only include check-ins at businesses present in our restaurant data frame.

In [32]:
checkin_df = checkin_df[checkin_df['business_id'].isin(restaurant_df['business_id'])].reset_index(drop=True)
checkin_df.head()

,business_id,date
0,-00d-Qb0q2TcWn-8LBHDZg,"2012-09-18 21:21:30, 2012-10-01 21:26:39, 2012..."
1,-0Gbsd7ztvTyFpl7jF0DIw,"2013-04-25 16:08:05, 2013-05-14 16:28:21, 2013..."
2,-0iqnv7MjKrgh7Q7bYRlUQ,"2011-02-09 19:13:57, 2011-02-12 18:32:40, 2011..."
3,-0xezDAvKicBZpVx9GU3Wg,"2011-09-05 18:40:44, 2011-10-30 22:54:12, 2012..."
4,-1UjMOUU9BGNOGj9KaT64w,"2010-02-02 19:34:27, 2010-05-18 11:07:54, 2010..."


In [33]:
checkin_df.shape

(10030, 2)

The resulting shape of this data frame is 10,030 rows with 2 columns.

## Load the Tip Dataset

We now load the tip dataset. We will use the chunks method again.

In [34]:
%%time
data_lines = []

with open("../data/tip.json", "r") as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                          chunksize=1000)
        
    for chunk in reader:
        data_lines.append(chunk)
    
tip_df = pd.concat(data_lines, ignore_index=True)

CPU times: user 8.64 s, sys: 302 ms, total: 8.95 s
Wall time: 8.95 s


In [35]:
tip_df.head()

,user_id,business_id,text,date,compliment_count
0,WCjg0jdHXMlwbqS9tZUx8Q,ENwBByjpoa5Gg7tKgxqwLg,Carne asada chips...,2011-07-22 19:07:35,0
1,42-Z02y9bABShAGZhuSzrQ,jKO4Og6ucdX2-YCTKQVYjg,Best happy hour from 3pm to 6pm! $1 off martin...,2014-09-10 07:33:29,0
2,5u7E3LYp_3eB8dLuUBazXQ,9Bto7mky640ocgezVKSfVg,"Nice people, skilled staff, clean location - b...",2013-12-13 23:23:41,0
3,wDWoMG5N9oI4DJ-p7z8EBg,XWFjKtRGZ9khRGtGg2ZvaA,"1/2-price bowling & the ""Very"" Old Fashion are...",2017-07-11 23:07:16,0
4,JmuFlorjjRshHTKzTwNtgg,mkrx0VhSMU3p3uhyJGCoWA,"Solid gold's. Great sauna. Great staff, too. E...",2016-11-30 08:46:36,0


In [36]:
tip_df.shape

(1162119, 5)

We now filter for tips related to only the business in our restaurant data frame.

In [37]:
tip_df = tip_df[tip_df['business_id'].isin(restaurant_df['business_id'])].reset_index(drop=True)
tip_df.head()

,user_id,business_id,text,date,compliment_count
0,fTsVDajAyDJ-YzsSdfXSDw,oQyf1788YWsiDLupGva6sw,Cold cuts are the best,2015-06-09 14:35:57,0
1,TOkW-RuqftALEh8E99UazQ,tdIyREwlre3SVU41a4eBgA,Clam chowder is fantastic!,2014-03-16 18:01:55,0
2,9D7pKuioWzigvboVqIcftQ,0895JZogvun_IO5fQqY6Ag,here's the greatest tip of all.. stay away fro...,2012-10-30 19:33:09,0
3,Atxahsei73oG3sMLhnPRaw,Tbq4hgX7uUzgFUWDlmqnBw,Don't get their cocktails. There is absolutely...,2012-05-23 21:18:48,0
4,t903_es-gp3abvdrIQutQA,W-u23r-Qf2OJ6MgFXeFm9A,Free parking in the rear. Great margaritas!,2010-05-16 01:19:47,0


In [38]:
tip_df.shape

(148977, 5)

The resulting shape of the tip data frame is 148,977 rows and 5 columns.

## Photo Captions

In [9]:
%%time
caption_df = pd.read_json('../data/photos.json', lines=True)
caption_df.head()

CPU times: user 699 ms, sys: 134 ms, total: 832 ms
Wall time: 833 ms


,photo_id,business_id,caption,label
0,Un_Og6jfhazVn7CxszkKEw,R1sIqGfKpF5f3HV3vyNsbg,Drinks! Front is beer obviously. The pink one ...,drink
1,BFE1AFOs27scnnfeBf99ZA,vdT7zlrLB2DL9pStDUs91A,,drink
2,7t-C0r1JRdoVD9FS7M-N7Q,c5GzrObEdj7eNVOdAxrppg,Stella on draft,drink
3,rLnw0d-YYZvT9kR4y7h7_Q,aQa7N5ZbPhCoKYGGB-gqfg,,drink
4,Cv5M8MDw8a5NEWvw2AQ4nw,B-s6qOFD75syhwWjp518aA,,drink


In [10]:
caption_df.shape

(200000, 4)

In [11]:
caption_df = caption_df[caption_df['business_id'].isin(restaurant_df['business_id'])].reset_index(drop=True)
caption_df.head()

,photo_id,business_id,caption,label
0,bnJDeS7YSX7jZM6pTsss2Q,1E2KcGtzZO5v_LgrTiQl9A,,drink
1,hW-4Q4Tk8uWz6YxBhvK-7A,A4js3H3Icd45DaaqfD5dQw,Sex on the beach,drink
2,Wre7MLQVlzYFk1F_xEao6Q,UXb5pTNPf31BpJ2oiDZBXQ,,drink
3,OT3yqNuH06nZln2utQogAg,xVxjzyMqghCYwVe_dwJzJw,"michelada\, I guess",drink
4,5oWMVtV136CYv_vjaGU-2A,A7oeXiStIt3uTU9OT07Qpw,Drink choices.,drink


In [12]:
caption_df.shape

(27053, 4)

We can now export our data frames to CSV files for use in upcoming notebooks for this project.

In [40]:
%%time
restaurant_df.to_csv('../data/ma_restaurant.csv', index=False)
review_df.to_csv('../data/ma_review.csv', index=False)
user_df.to_csv('../data/ma_user.csv', index=False)
checkin_df.to_csv('../data/ma_checkin.csv', index=False)
tip_df.to_csv('../data/ma_tip.csv', index=False)


CPU times: user 32.6 s, sys: 1.62 s, total: 34.2 s
Wall time: 35.1 s
